In [262]:
import itertools
import multiprocessing

import numba
import numpy as np

from nltk import tokenize
from nltk.util import ngrams

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_distances

In [331]:
@numba.njit
def jaccard_similarity(a, b):
    union = np.sum(a | b)
    if not union:
        return 1.0
    return np.sum(a & b) / union


@numba.njit
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


@numba.njit
def cohesion(chromosome, similarity, document):
    total = 0
    for p in np.unique(chromosome):
        sents = document[chromosome == p]
        k = len(sents)
        #: combinations choose 2
        for i in range(k-1):
            for j in range(i+1, k):
                total += similarity(sents[i], sents[j]) / len(sents)  
    return total


@numba.njit
def separation(chromosome, similarity, document):
    total = 0
    k = len(np.unique(chromosome))
    #: combinations choose 2
    for p in range(k-1):
        for q in range(p+1, k):
            sents_p = document[chromosome == p]
            sents_q = document[chromosome == q]
            #: product
            m, n = len(sents_p), len(sents_q)
            for i in range(m):
                for j in range(n):
                    total += similarity(sents_p[i], sents_q[j]) / m / n
    return total


@numba.njit
def cohesion_separation(chromosome, similarity, document):
    coh = cohesion(chromosome, similarity, document)
    sep = separation(chromosome, similarity, document)
    return (1 + sigmoid(coh)) ** sep

In [324]:
def init_chromosome(choices, length):
    chrom = np.full(length, -1)
    #: ensure that each choice is accounted for at least once
    idxs = np.random.choice(np.arange(length), len(choices), replace=False)
    chrom[idxs] = np.random.permutation(choices)
    idxs = np.where(chrom == -1)[0]
    chrom[idxs] = np.random.choice(choices, len(idxs))
    return chrom


def init_population(population_size, cluster_amount, chromosome_length):
    clusts = np.arange(cluster_amount)
    chroms = [init_chromosome(clusts, chromosome_length) for _ in range(population_size)]
    pop = np.vstack(chroms)
    return pop


def get_offspring_distinct(population, randoms, lambda_, crossover_rate):
    n = np.arange(len(population))
    s = set(n)
    idxs = np.array([np.random.choice(tuple(s - {i}), size=3, replace=False) for i in n])
    chrom_1, chrom_2, chrom_3 = map(np.squeeze, np.split(population[idxs], 3, axis=1))
    k = len(np.unique(population))
    offspr = (chrom_1 + lambda_ * (chrom_2 - chrom_3)) % k
    mask = randoms < crossover_rate
    offspr[mask] = population[mask]
    return offspr


def get_offspring(population, randoms, lambda_, crossover_rate):
    #: For computation time, relax requirement that X_r, X_r1, X_r2, X_r3 are distinct. 
    #: With large population size, this is unlikely to occur, and if it does, it doesn't
    #: seem that detrimental. Also is this mitigated with appropriate lam choice?
    n = len(population)
    idxs = np.random.choice(np.arange(n), size=(n, 3))
    chrom_1, chrom_2, chrom_3 = map(np.squeeze, np.split(population[idxs], 3, axis=1))
    k = len(np.unique(population))
    offspr = (chrom_1 + lambda_ * (chrom_2 - chrom_3)) % k
    mask = randoms < crossover_rate
    offspr[mask] = population[mask]
    return offspr


# def next_generation(population, offspring, func):
#     fit_off = np.array([func(chrom) for chrom in offspring])
#     fit_pop = np.array([func(chrom) for chrom in population])
#     mask = fit_off > fit_pop
#     population[mask] = offspring[mask]
#     return


def next_generation(population, offspring, func, pool):
    fits = pool.map(func, itertools.chain(population, offspring))
    l = len(population)
    fit_pop = np.array(fits[:l])
    fit_off = np.array(fits[l:])
    mask = fit_off > fit_pop
    population[mask] = offspring[mask]
    return


def mutate(population, randoms):
    mask = randoms < sigmoid(population)
    #: inversion operator
    idxs = np.nonzero(mask)
    arr = np.array(idxs)
    sorter = np.lexsort((-arr[1], arr[0]))
    rev = arr.T[sorter].T
    population[idxs] = population[(rev[0], rev[1])]
    return

In [325]:
#TODO: early stopping --> little fitness improvement over x generations, good enough fitness score
def run_iterations(pop_size, summ_len, num_sents, func, lam, cr, iterations, *, mutate_after=True,
                   seed=None, verbose=False, save_rate=np.nan, save_dir=None):
    
    if save_dir is not None:
        save_dir = pathlib.Path(save_dir)
        if not save_dir.is_dir():
            msg = f'save_dir={save_dir} not a valid directory path'.format(save_dir=save_dir)
            raise NotADirectoryError(msg)
    
    if seed is not None:
        np.random.seed(seed)
        
    if isinstance(summ_len, int):
        if not (0 < summ_len < num_sents):
            raise ValueError('int summ_len must be between 0 and the number of sentences in the document')
    elif isinstance(summ_len, float):
        if not (0.0 < summ_len < 1.0):
            raise ValueError('float summ_len must be between 0.0 and 1.0')
        summ_len = int(summ_len * num_sents)
    else:
        raise TypeError('summ_len must be a float or int')
        
    if pop_size < 10:
        raise ValueError('pop_size must be at least 10')
    pop_size = int(pop_size)
    
    
    
    pop = init_population(pop_size, summ_len, num_sents)
    shape = pop.shape
    pool = multiprocessing.Pool()
    
    for i in range(iterations):
        if i % save_rate == 0:
            file = save_dir / 'generation_{i:0>pad}'.format(i=i, pad=len(str(iterations)))
            np.save(file, pop)
            
        if verbose:
            print(i)  #TODO: logfile --> iteration number, best fitness score, avg fitness score, hyper-params
        
        rand = np.random.random_sample(shape)
        
        t0 = time.time()
        offspr = get_offspring_distinct(pop, rand, lam, cr)
        t1 = time.time()
        PROFILER['offspring'] += t1 - t0
        
        #: option since papers unclear if mutate at offspring or survivors stage
        if not mutate_after:
            mutate(offspr, rand)
            
        t0 = time.time()
        next_generation(pop, offspr, func, pool)
#         next_generation(pop, offspr, func)
        t1 = time.time()
        PROFILER['generation'] += t1 - t0
        
        if mutate_after:
            t0 = time.time()
            mutate(pop, rand)
            t1 = time.time()
            PROFILER['mutate'] += t1 - t0
    
    pool.terminate()
    return pop

In [285]:
def best_chromosome(population):
    #TODO: make sure it picks one with all k-clusters
    fits = np.argmax([fitness(chrom) for chrom in population])
    chrom = population[fits]
    return chrom
    

def central_sentences(chromosome, document, metric=cosine_distances):
    central_sents = []
    for cluster in np.unique(chromosome):
        idxs = np.where(chromosome == cluster)[0]
        sents = document[idxs]
        centroid = sents.mean(axis=0)[np.newaxis,:]
        dists = metric(sents, centroid)
        cent_sent = idxs[np.argmin(dists)]
        central_sents.append(cent_sent)
    return sorted(central_sents)

In [286]:
def rouge_n(n, y_pred, y_true):
    n_gram_pred = set(ngrams(y_pred, n))
    n_gram_true = set(ngrams(y_true, n))
    return len(n_gram_pred & n_gram_true) / len(n_gram_true)

# Run Algorithm

In [287]:
path = cwd / 'the_raven.txt'
with open(path) as fp:
    text = fp.read()
text[:100]

'  Once upon a midnight dreary, while I pondered, weak and weary,\n  Over many a quaint and curious vo'

In [318]:
import json
import pathlib

cwd = pathlib.Path.cwd()
data = cwd / 'data'
jsons = data / 'jsons'
json_2018 = jsons / '2018' / '2018.json'

with open(json_2018) as fp:
    articles_2018 = json.load(fp)['2018']

article = articles_2018[863]
text = article['story']
summ_true = article['summary']
# print(text)

In [319]:
print(text)

President Trump is throwing his support behind legislation that could shorten sentences for some drug offenders and help prisoners adjust to life after incarceration.

Details of the measure have not been officially released, but Trump said Wednesday the bill will provide incentives for prisoners to participate in training or rehabilitation programs with a goal of reducing recidivism.

It will also include measures to address sentencing disparities and inequities.

"Americans across the political spectrum can unite around prison reform legislation that will reduce crime, while giving our fellow citizens a chance at redemption," the president said.

Trump added that he is ready to sign the bill into law whenever it reaches his desk.

Legislation that passed the House of Representatives in May included measures targeting recidivism after incarceration, but did not include sentencing provisions.

That was a sticking point for Republican Sen. Chuck Grassley, who helms the Senate Judiciary 

In [320]:
summ_true

' The president\'s support is a big win for activists who have been pushing for criminal justice measures that roll back the stiff penalties imposed as a part of America\'s decades-long "war on drugs."'

In [330]:
import time
import collections


PROFILER = collections.Counter()


def fitness(chromosome):
    return cohesion_separation(chromosome, jaccard_similarity, doc)

cv = CountVectorizer(stop_words='english')
sents_lower = tokenize.sent_tokenize(text.lower())
# sents_lower = [t.strip() for t in tokenize.line_tokenize(text.lower())]
vec = cv.fit_transform(sents_lower)
doc = vec.toarray().astype(bool).astype(int)

ratio = 0.1

t0 = time.time()
pop = run_iterations(pop_size=100, summ_len=ratio, num_sents=len(doc), 
                     func=fitness, lam=0.9, cr=0.5, iterations=1000, verbose=False, seed=0)
t1 = time.time()
print(t1-t0)


chrom_best = best_chromosome(pop)
idxs = central_sentences(chrom_best, doc)
#: 2018 article[999] -- gay skier winter olympics -> no lower() causes wierd sentence breaks that throw off summary
# sents_norm = tokenize.sent_tokenize(text)
summ_evol = '\n'.join(np.array(sents_lower)[idxs])

PROFILER

24.836180210113525


Counter({'offspring': 3.056461811065674,
         'generation': 21.168840169906616,
         'mutate': 0.3959167003631592})

In [327]:
# get the uppercase correct
summ_evol = []
for sent in np.array(sents_lower)[idxs]:
    start = text.lower().index(sent)
    stop = start + len(sent)
    summ_evol.append(text[start:stop])
summ_evol = '\n'.join(summ_evol)
print(summ_evol)

Grassley, who joined Trump at the White House for Wednesday's announcement, has been working with Sen. Dick Durbin, D-Ill., and others to put forward legislation that addresses sentencing and changes prison policies.
Trump's endorsement is critical for getting the Republican support necessary to get a bill passed during the lame-duck session, said Mark Holden, general counsel to Koch Industries, which has been pushing for the changes.
In a tweet after the announcement, the president said members of the House and Senate "have poured their time, heart and energy into the crucial issue."


In [323]:
# get the uppercase correct
summ_evol = []
for sent in np.array(sents_lower)[idxs]:
    start = text.lower().index(sent)
    stop = start + len(sent)
    summ_evol.append(text[start:stop])
summ_evol = '\n'.join(summ_evol)
print(summ_evol)

Grassley, who joined Trump at the White House for Wednesday's announcement, has been working with Sen. Dick Durbin, D-Ill., and others to put forward legislation that addresses sentencing and changes prison policies.
Trump's endorsement is critical for getting the Republican support necessary to get a bill passed during the lame-duck session, said Mark Holden, general counsel to Koch Industries, which has been pushing for the changes.
In a tweet after the announcement, the president said members of the House and Senate "have poured their time, heart and energy into the crucial issue."


In [234]:
def scores(n, y_pred):
    for i in range(1, n+1):
        score = rouge_n(i, y_pred, summ_true)
        print(i, score)

In [235]:
print('evolutionary')
scores(3, summ_evol)

evolutionary
1 0.8571428571428571
2 0.7222222222222222
3 0.3735632183908046


In [236]:
import gensim

summ_gensim = gensim.summarization.summarize(text, ratio=ratio)
print('gensim')
scores(3, summ_gensim)

gensim
1 0.8857142857142857
2 0.7142857142857143
3 0.367816091954023


In [237]:
print(summ_gensim)

The vice speaker of Iran's parliament also told reporters that the U.S. withdrawal from the 2015 deal, known as the Joint Comprehensive Plan of Action, made new negotiations a nonstarter.
Kenyon reports that Rouhani himself — who has not publicly responded to the invitation — told parliament that the U.S. has proven it doesn't keep its promises and that Iran will protect its right to export oil, which is the target of U.S. sanctions.


In [238]:
import string

from sumy.parsers.plaintext import PlaintextParser
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.nlp.tokenizers import Tokenizer



def format_sumy(sumy_summary):
    return ''.join(char for sent in sumy_summary for char in str(sent) if char in string.printable)

parser = PlaintextParser(text, Tokenizer('english'))

text_rank = TextRankSummarizer()
summ_text_rank = format_sumy(text_rank(parser.document, len(idxs)))
print('sumy - text rank')
scores(3, summ_text_rank)

print()

lex_rank = LexRankSummarizer()
summ_lex_rank = format_sumy(lex_rank(parser.document, len(idxs)))
print('sumy - lex rank')
scores(3, summ_lex_rank)

sumy - text rank
1 0.8571428571428571
2 0.7063492063492064
3 0.3620689655172414

sumy - lex rank
1 0.8
2 0.6587301587301587
3 0.39655172413793105


In [239]:
print(summ_text_rank)

Last week, after Rouhani said that "peace with Iran is the mother of all peace, and war with Iran is the mother of all wars," Trump responded with an all-caps, late-night tweet warning of "CONSEQUENCES THE LIKES OF WHICH FEW THROUGHOUT HISTORY HAVE EVER SUFFERED BEFORE" if Iran should "THREATEN THE UNITED STATES AGAIN."Kenyon reports that Rouhani himself  who has not publicly responded to the invitation  told parliament that the U.S. has proven it doesn't keep its promises and that Iran will protect its right to export oil, which is the target of U.S. sanctions.


In [240]:
print(summ_lex_rank)

Kenyon reports that Rouhani himself  who has not publicly responded to the invitation  told parliament that the U.S. has proven it doesn't keep its promises and that Iran will protect its right to export oil, which is the target of U.S. sanctions.Kenyon says there will be some pressure on Iran to negotiate with the U.S.  but he puts that in perspective.
